# Описание
Источник: https://github.com/Grossmend/DialoGPT

Статья на хабре https://habr.com/ru/company/icl_services/blog/548244/


Модель доступна на Hugging Face Model Hub https://huggingface.co/Grossmend/rudialogpt3_medium_based_on_gpt2 . Также можете скачать с Google Drive https://drive.google.com/file/d/1FCwByq-VkeW1_cje4sIB5KaY0S9a8u8b/view?usp=sharing.

Описание как использовать https://huggingface.co/Grossmend/rudialogpt3_medium_based_on_gpt2


In [1]:
#!pip install torch==1.6.0 torchvision==0.7.0 # cudatoolkit=10.2 # torch==1.8.1 (current)
!pip install transformers==4.4.2

     |████████████████████████████████| 2.0MB 9.1MB/s 
     |████████████████████████████████| 901kB 46.3MB/s 
     |████████████████████████████████| 3.3MB 49.5MB/s 


In [2]:
import os
path_model='/content/RuDialoGPT/model'
if not os.path.exists(path_model):
  if not os.path.exists("/content/RuDialoGPT.zip"):
    !gdown --id 1FCwByq-VkeW1_cje4sIB5KaY0S9a8u8b
  !unzip /content/RuDialoGPT.zip

# Проверяем, что все на месте
if not os.path.isdir(path_model):
  raise Exception(f"Path '{path_model}' not found!")
if not os.path.isfile(os.path.join(path_model, 'config.json')):
  raise Exception(f"On path '{path_model}' file 'config.json' not found!")
if not os.path.isfile(os.path.join(path_model, 'pytorch_model.bin')):
  raise Exception(f"On path '{path_model}' file 'pytorch_model.bin' not found!")
if not os.path.isfile(os.path.join(path_model, 'vocab.json')):
  raise Exception(f"On path '{path_model}' file 'vocab.json' not found!")
if not os.path.isfile(os.path.join(path_model, 'tokenizer_config.json')):
  raise Exception(f"On path '{path_model}' file 'tokenizer_config.json' not found!")
if not os.path.isfile(os.path.join(path_model, 'special_tokens_map.json')):
  raise Exception(f"On path '{path_model}' file 'special_tokens_map.json' not found!")
if not os.path.isfile(os.path.join(path_model, 'merges.txt')):
  raise Exception(f"On path '{path_model}' file 'merges.txt' not found!")

Downloading...
From: https://drive.google.com/uc?id=1FCwByq-VkeW1_cje4sIB5KaY0S9a8u8b
To: /content/RuDialoGPT.zip
1.32GB [00:16, 81.3MB/s]
Archive:  /content/RuDialoGPT.zip
   creating: RuDialoGPT/
  inflating: RuDialoGPT/talk_with_model.ipynb  
   creating: RuDialoGPT/model/
  inflating: RuDialoGPT/model/tokenizer_config.json  
  inflating: RuDialoGPT/model/config.json  
  inflating: RuDialoGPT/model/pytorch_model.bin  
  inflating: RuDialoGPT/model/merges.txt  
  inflating: RuDialoGPT/model/special_tokens_map.json  
  inflating: RuDialoGPT/model/vocab.json  


In [3]:
# Клонируем модель (но не скачиваем сами файлы модели)
!git clone https://huggingface.co/Grossmend/rudialogpt3_medium_based_on_gpt2
!ls -l rudialogpt3_medium_based_on_gpt2/

Cloning into 'rudialogpt3_medium_based_on_gpt2'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 36 (delta 14), reused 0 (delta 0)
Unpacking objects: 100% (36/36), done.
total 2944
-rw-r--r-- 1 root root     827 Jun  4 22:32 config.json
-rw-r--r-- 1 root root     135 Jun  4 22:32 flax_model.msgpack
-rw-r--r-- 1 root root 1270925 Jun  4 22:32 merges.txt
-rw-r--r-- 1 root root     135 Jun  4 22:32 pytorch_model.bin
-rw-r--r-- 1 root root    1891 Jun  4 22:32 README.md
-rw-r--r-- 1 root root     275 Jun  4 22:32 special_tokens_map.json
-rw-r--r-- 1 root root     549 Jun  4 22:32 tokenizer_config.json
-rw-r--r-- 1 root root 1713123 Jun  4 22:32 vocab.json


In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(path_model)
model = AutoModelForCausalLM.from_pretrained(path_model)

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [10]:
# Функция определения длины диалога
def get_length_param(text: str, tokenizer) -> str:
    tokens_count = len(tokenizer.encode(text))
    if tokens_count <= 15:
        len_param = '1'
    elif tokens_count <= 50:
        len_param = '2'
    elif tokens_count <= 256:
        len_param = '3'
    else:
        len_param = '-'
    return len_param

In [ ]:
# Тестирование 5ть циклов на параметрах по умолчанию
for step in range(5):
    
    input_user = input("===> User:")
    
    # encode the new user input, add parameters and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(f"|0|{get_length_param(input_user)}|" + input_user + tokenizer.eos_token +  "|1|1|", return_tensors="pt")

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids
    
    # generated a response
    chat_history_ids = model.generate(
        bot_input_ids,
        num_return_sequences=1,
        max_length=512,
        no_repeat_ngram_size=3,
        do_sample=True,
        top_k=50,
        top_p=0.9,
        temperature = 0.6,
        mask_token_id=tokenizer.mask_token_id,
        eos_token_id=tokenizer.eos_token_id,
        unk_token_id=tokenizer.unk_token_id,
        pad_token_id=tokenizer.pad_token_id,
        device='cuda', #device='cpu',
    )
    
    # pretty print last ouput tokens from bot
    print(f"===> RuDialoGPT: {tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)}")

===> User:Привет!
===> RuDialoGPT: И тебе привет!
===> User:Вежливый бот?
===> RuDialoGPT: Напоминает
===> User:Что напоминает?
===> RuDialoGPT: Бот
===> User:Бот напоминает бота? Это тавтологоия
===> RuDialoGPT: Это что-то вроде "ты мне не друг, приятель"
===> User:Умный?
===> RuDialoGPT: Я не знаю, что это.


In [7]:
#@title Параметры модели

# Поле 'Max length tokens generate' должно быть больше 8, но меньше 256
max_length = 256 #@param {type:"integer"}
# def 3 Поле 'No repeat ngram size' должно быть больше или равно 1, но меньше или равно 10.
no_repeat_ngram_size = 3 #@param {type:"integer"}
# def 100 Поле 'Top K' должно быть больше или равно 1, но меньше или равно 500. Текущее значение: 
top_k = 100 #@param {type:"integer"}
# def 0.9 Поле 'Top P' должно быть больше или равно 0.01, но меньше или равно 1.0.
top_p = 0.9 #@param {type:"number"}
# def 0.6 Поле 'Temperature' должно быть больше или равно 0.01, но меньше или равно 1.0. 
temperature = 0.6 #@param {type:"number"}
# def 5. Поле 'Num responses return' должно быть больше или равно 1, но меньше или равно 10. 
num_return = 5 #@param {type:"integer"}
# def True
do_sample = True  #@param {type:"boolean"}
# def True
is_always_use_length = True  #@param {type:"boolean"}
# Поле 'Length generate' должно принимать одно из следующих значений: [0, 1, 2, 3].
length_gen = 1  #@param {type:"integer"}
# def False. debug
log_debug = True  #@param {type:"boolean"}

params = { 
        "max_length": max_length,
        "no_repeat_ngram_size": no_repeat_ngram_size, 
        "top_k": top_k,                              
        "top_p": top_p,                              
        "temperature": temperature,                   
        "num_return_sequences": num_return,           
        "do_sample": do_sample,
        "device": 'cuda', #'cuda:0', # 'cpu',
        "is_always_use_length": is_always_use_length,
        "length_generate": length_gen, 
        "log_debug": log_debug
    }

In [8]:
# Генерация ответа
def generate(full_inputs, params): # inputs = [{'speaker': 0, 'text': 'Привет, как день прошел?'},{'speaker': 1, 'text': 'Хорошо, а у тебя как?'}]
  if len(full_inputs) > 5: # Ограничиваем кол-во истории в диалоге на 2 предыдущих + вопрос
    inputs = full_inputs[-5:]
  else:
    inputs = full_inputs
  inputs_text = ''
  for input_ in inputs:
    if params['is_always_use_length']:
        length_param = get_length_param(input_['text'], tokenizer)
    else:
        length_param = '-'
    inputs_text += f"|{input_['speaker']}|{length_param}|{input_['text']}"
  inputs_text += f"|1|{params['length_generate']}|"

  if params['log_debug']:
    print(f"        [debug] Затравка вопроса для модели: {inputs_text}")
  inputs_token_ids = tokenizer.encode(inputs_text, return_tensors='pt')

  try:
    # ToDo make this asynchronous
    outputs_token_ids = model.generate(
        inputs_token_ids,
        max_length=params['max_length'],
        no_repeat_ngram_size=params['no_repeat_ngram_size'],
        do_sample=params['do_sample'],
        top_k=params['top_k'],
        top_p=params['top_p'],
        temperature=params['temperature'],
        num_return_sequences=params['num_return_sequences'],
        device=params['device'],
        mask_token_id=tokenizer.mask_token_id,
        eos_token_id=tokenizer.eos_token_id,
        unk_token_id=tokenizer.unk_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )
  except Exception as e:
    print(f"===> Error generate: {str(e)}")
    return {'inputs': '', 'outputs': '', 'status': False, 'msg': f"{str(e)}"}

  outputs = [tokenizer.decode(x, skip_special_tokens=True) for x in outputs_token_ids]
  outputs = [x.split('|')[-1] for x in outputs]

  return {'inputs': inputs, 'outputs': outputs, 'status': True, 'msg': ''}

In [11]:
# Диалог с чатоботом
print('Введите quit для остановки')
if params['log_debug']:
  print(f"\n===> debug Params generate: {params}\n\n")
inputs = []
while True:
  user_input = input("USER:")
  if user_input == "quit":
    "stop talking!"
    break
  inputs.append({'speaker': 0, 'text': user_input})
  response_data = generate(inputs, params)
  bot_resposnse = response_data['outputs'][0]
  variants_responses = response_data['outputs']
  
  if not response_data['status']:
    print("BOT (ошибка):", response_data['msg'])
  else:
    bot_resposnse = response_data['outputs'][0]
    inputs.append({'speaker': 1, 'text': bot_resposnse})
    variants_responses = response_data['outputs']
    print("\n>>>BOT:", bot_resposnse)
    if params['log_debug']:
     print("        [debug] other variants ===> :", variants_responses[1:])
     

Введите quit для остановки

===> debug Params generate: {'max_length': 256, 'no_repeat_ngram_size': 3, 'top_k': 100, 'top_p': 0.9, 'temperature': 0.6, 'num_return_sequences': 5, 'do_sample': True, 'device': 'cuda', 'is_always_use_length': True, 'length_generate': 1, 'log_debug': True}


USER:Привет!
        [debug] Затравка вопроса для модели: |0|1|Привет!|1|1|

>>>BOT: И тебе привет, соседушка
        [debug] other variants ===> : ['Вай, спасибо большое', 'Братан, привет', 'И тебе привет)', 'Здравствуй!']


KeyboardInterrupt: ignored